# Stanford RNA 3D Folding - Template-Based Approach
Template-based modeling using sequence homology and structure threading.

In [ ]:
import numpy as np
import pandas as pd
from typing import List, Dict, Tuple

## Sequence Alignment

In [ ]:
def needleman_wunsch_align(
    seq1: str,
    seq2: str,
    match_score: int = 2,
    mismatch_penalty: int = -1,
    gap_penalty: int = -2
) -> Tuple[str, str, float]:
    """Global sequence alignment using Needleman-Wunsch algorithm."""
    n, m = len(seq1), len(seq2)

    # Initialize scoring matrix
    score_matrix = np.zeros((n + 1, m + 1))
    score_matrix[0, :] = np.arange(m + 1) * gap_penalty
    score_matrix[:, 0] = np.arange(n + 1) * gap_penalty

    # Fill scoring matrix
    for i in range(1, n + 1):
        for j in range(1, m + 1):
            match = score_matrix[i-1, j-1] + (
                match_score if seq1[i-1] == seq2[j-1] else mismatch_penalty
            )
            delete = score_matrix[i-1, j] + gap_penalty
            insert = score_matrix[i, j-1] + gap_penalty
            score_matrix[i, j] = max(match, delete, insert)

    # Traceback
    align1, align2 = [], []
    i, j = n, m

    while i > 0 or j > 0:
        current_score = score_matrix[i, j]

        if i > 0 and j > 0:
            diagonal_score = score_matrix[i-1, j-1] + (
                match_score if seq1[i-1] == seq2[j-1] else mismatch_penalty
            )
            if abs(current_score - diagonal_score) < 1e-6:
                align1.append(seq1[i-1])
                align2.append(seq2[j-1])
                i -= 1
                j -= 1
                continue

        if i > 0:
            up_score = score_matrix[i-1, j] + gap_penalty
            if abs(current_score - up_score) < 1e-6:
                align1.append(seq1[i-1])
                align2.append('-')
                i -= 1
                continue

        if j > 0:
            left_score = score_matrix[i, j-1] + gap_penalty
            if abs(current_score - left_score) < 1e-6:
                align1.append('-')
                align2.append(seq2[j-1])
                j -= 1
                continue

        break

    align1 = ''.join(reversed(align1))
    align2 = ''.join(reversed(align2))
    alignment_score = score_matrix[n, m]

    return align1, align2, alignment_score

## Template Finding

In [ ]:
def find_best_template(
    query_seq: str,
    template_library: List[Dict],
    top_k: int = 5
) -> List[Dict]:
    """Find best template structures for a query sequence."""
    candidates = []

    for template in template_library:
        template_seq = template['sequence']

        # Skip empty templates
        if len(template_seq) == 0:
            continue

        # Quick filter: length difference
        length_ratio = len(query_seq) / len(template_seq)
        if length_ratio < 0.5 or length_ratio > 2.0:
            continue

        # Align sequences
        aligned_query, aligned_template, align_score = needleman_wunsch_align(
            query_seq, template_seq
        )

        # Compute identity
        identity = sum(1 for a, b in zip(aligned_query, aligned_template) if a == b and a != '-')
        identity /= len(aligned_query)

        candidates.append({
            'template': template,
            'aligned_query': aligned_query,
            'aligned_template': aligned_template,
            'identity': identity,
            'align_score': align_score,
        })

    # Sort by identity (descending)
    candidates.sort(key=lambda x: x['identity'], reverse=True)

    return candidates[:top_k]

## Structure Threading

In [ ]:
def thread_sequence_on_template(
    query_seq: str,
    aligned_query: str,
    aligned_template: str,
    template_coords: np.ndarray
) -> np.ndarray:
    """Thread query sequence onto template structure."""
    query_coords = []
    template_pos = 0
    query_pos = 0

    for i, (q_res, t_res) in enumerate(zip(aligned_query, aligned_template)):
        if q_res != '-':
            if t_res != '-':
                # Both aligned: copy template coordinates
                query_coords.append(template_coords[template_pos])
                template_pos += 1
            else:
                # Insertion in query: interpolate
                if len(query_coords) > 0 and template_pos < len(template_coords):
                    prev_coord = query_coords[-1]
                    next_coord = template_coords[template_pos]
                    interpolated = (prev_coord + next_coord) / 2.0
                    query_coords.append(interpolated)
                elif len(query_coords) > 0:
                    # Extend from last known position
                    query_coords.append(query_coords[-1] + np.array([3.8, 0.0, 0.0]))
                else:
                    # Start with first template coordinate
                    query_coords.append(template_coords[0])
            query_pos += 1
        else:
            # Deletion in query: skip template position
            if t_res != '-':
                template_pos += 1

    return np.array(query_coords)

## Ensemble Generation

In [ ]:
def generate_ensemble_from_template(
    query_seq: str,
    template_info: Dict,
    num_variants: int = 5
) -> List[np.ndarray]:
    """Generate ensemble of structures from a template by perturbation."""
    template = template_info['template']
    aligned_query = template_info['aligned_query']
    aligned_template = template_info['aligned_template']

    # Get base structure
    base_coords = thread_sequence_on_template(
        query_seq,
        aligned_query,
        aligned_template,
        template['coordinates']
    )

    # Generate variants with increasing noise
    variants = []
    noise_levels = np.linspace(0.5, 1.4, num_variants)

    for noise_std in noise_levels:
        perturbed = base_coords + np.random.randn(*base_coords.shape) * noise_std
        variants.append(perturbed)

    return variants

## Main Prediction Function

In [ ]:
def predict_structure_template_based(
    query_seq: str,
    template_library: List[Dict],
    num_predictions: int = 5
) -> List[np.ndarray]:
    """Predict RNA structure using template-based approach."""
    # Find best templates
    top_templates = find_best_template(query_seq, template_library, top_k=3)

    if not top_templates:
        # Fallback: random coil
        L = len(query_seq)
        predictions = []
        for _ in range(num_predictions):
            coords = np.zeros((L, 3))
            coords[:, 0] = np.arange(L) * 3.8
            coords += np.random.randn(L, 3) * 2.0
            predictions.append(coords)
        return predictions

    # Use best template
    best_template = top_templates[0]

    # Generate ensemble
    predictions = generate_ensemble_from_template(
        query_seq, best_template, num_variants=num_predictions
    )

    return predictions

## Build Template Library from Training Data

In [ ]:
# Load training data
train_df = pd.read_csv('/kaggle/input/stanford-rna-3d-folding-2/train_labels.csv')

print("Building template library from training data...")
template_library = []

for target_id in train_df['target_id'].unique():
    target_data = train_df[train_df['target_id'] == target_id].sort_values('resid')
    
    # Extract sequence
    sequence = ''.join(target_data['resname'].values)
    
    # Extract coordinates
    coords = target_data[['x', 'y', 'z']].values
    
    # Only add if we have complete coordinates
    if not np.isnan(coords).any() and len(sequence) > 0:
        template_library.append({
            'target_id': target_id,
            'sequence': sequence,
            'coordinates': coords
        })

print(f"Loaded {len(template_library)} templates")

## Generate Predictions for Test Set

In [ ]:
# Load test sequences
test_df = pd.read_csv('/kaggle/input/stanford-rna-3d-folding-2/test_sequences.csv')

print(f"Generating predictions for {len(test_df)} test sequences...")
submission_rows = []

for idx, row in test_df.iterrows():
    target_id = row['target_id']
    sequence = row['sequence']
    
    print(f"  Processing {target_id} (length {len(sequence)})...")
    
    # Predict structures
    predictions = predict_structure_template_based(
        sequence,
        template_library,
        num_predictions=5
    )
    
    # Convert to submission format
    for res_idx, nucleotide in enumerate(sequence):
        resid = res_idx + 1
        row_id = f"{target_id}_{resid}"
        
        # Collect coordinates from all 5 predictions
        coords_dict = {'ID': row_id, 'resname': nucleotide, 'resid': resid}
        
        for pred_num in range(5):
            pred_coords = predictions[pred_num][res_idx]
            coords_dict[f'x_{pred_num+1}'] = pred_coords[0]
            coords_dict[f'y_{pred_num+1}'] = pred_coords[1]
            coords_dict[f'z_{pred_num+1}'] = pred_coords[2]
        
        submission_rows.append(coords_dict)

print("Predictions complete!")

## Create Submission File

In [ ]:
# Create submission DataFrame
submission_df = pd.DataFrame(submission_rows)

# Ensure column order
columns = ['ID', 'resname', 'resid']
for i in range(1, 6):
    columns.extend([f'x_{i}', f'y_{i}', f'z_{i}'])

submission_df = submission_df[columns]

# Save submission
submission_df.to_csv('submission.csv', index=False)

print(f"Submission created!")
print(f"  Total rows: {len(submission_df):,}")
print(f"  Targets: {len(test_df)}")
print(f"\nFirst few rows:")
print(submission_df.head(10))